In [1]:
import os
import numpy as np
import scipy as sp
import numba as nb

import ROOT as rt
from ROOT import TCanvas, TH1D, TH2D, TGraph, TGraphErrors, TLegend, TLatex, TLine, TBox, RDataFrame

#####
rt.EnableImplicitMT(4)

gc = []
std_color_list = [1, 2, 4, 8, 6, 28, 43, 7, 25, 36, 30, 40, 42, 49, 46, 38, 32, 800, 600, 900, 870, 840]

#####

DATA_DIR = "/home/psimmerl/mds_analysis/data"

fn_signal = f"{DATA_DIR}/raw/mc_pedro.root"
fn_data = f"{DATA_DIR}/raw/data_pedro.root"




Welcome to JupyROOT 6.30/04


In [ ]:
MAX_CSC_JET_PT = 30
MAX_DT_JET_PT = 30

MAX_CSC_MUON_PT = 10
MAX_DT_MUON_PT = 10

MIN_DNN = 0.96

################

rdfs = {
    "signal" : RDataFrame("MuonSystem", fn_signal),
    "data" : RDataFrame("MuonSystem", fn_data),
}

################
C, D = "cscRechitCluster", "dtRechitCluster"

csc_gllp_req = """{C}_match_gLLP
&& (abs({C}_match_gLLP_eta) < 3)
&& ({C}_match_gLLP_decay_r < 800)
&& (400 < abs({C}_match_gLLP_decay_z))
&& (abs({C}_match_gLLP_decay_z) < 1200)""".replace("{C}", C)

dt_gllp_req = """{D}_match_gLLP
&& (200 < {D}_match_gLLP_decay_r)
&& ({D}_match_gLLP_decay_r < 800)
&& (abs({D}_match_gLLP_decay_z) < 700)""".replace("{D}", D)

l1_plateau_req = """
auto me11 = (100 < {C}R) && ({C}R < 275) && ( 580 < abs({C}Z)) && (abs({C}Z) <  632) && (500 <= {C}Size);
auto me12 = (275 < {C}R) && ({C}R < 465) && ( 668 < abs({C}Z)) && (abs({C}Z) <  724) && (200 <= {C}Size);
auto me13 = (505 < {C}R) && ({C}R < 700) && ( 668 < abs({C}Z)) && (abs({C}Z) <  724) && (200 <= {C}Size);
auto me21 = (139 < {C}R) && ({C}R < 345) && ( 789 < abs({C}Z)) && (abs({C}Z) <  850) && (500 <= {C}Size);
auto me22 = (357 < {C}R) && ({C}R < 700) && ( 791 < abs({C}Z)) && (abs({C}Z) <  850) && (200 <= {C}Size);
auto me31 = (160 < {C}R) && ({C}R < 345) && ( 915 < abs({C}Z)) && (abs({C}Z) <  970) && (500 <= {C}Size);
auto me32 = (357 < {C}R) && ({C}R < 700) && ( 911 < abs({C}Z)) && (abs({C}Z) <  970) && (200 <= {C}Size);
auto me41 = (178 < {C}R) && ({C}R < 345) && (1002 < abs({C}Z)) && (abs({C}Z) < 1063) && (500 <= {C}Size);
auto me42 = (357 < {C}R) && ({C}R < 700) && (1002 < abs({C}Z)) && (abs({C}Z) < 1063) && (200 <= {C}Size);
return ( me11 || me12 || me13 || me21 || me22 || me31 || me32 || me41 || me42 );
""".replace("{C}", C)

################
for key, rdf in rdfs.items():
    count, weight = rdf.Count(), rdf.Sum("weight")
    print(f"{key:>6}: N events = {count.GetValue():>10,} (weight = {weight.GetValue():>13,.2f})")

    rdf = rdf.Define(f"{C}R", f"ROOT::VecOps::sqrt( {C}X*{C}X +{C}Y*{C}Y )")
    rdf = rdf.Define(f"{D}R", f"ROOT::VecOps::sqrt( {D}X*{D}X +{D}Y*{D}Y )")
    rdf = rdf.Define(f"{C}ME1", f"{C}NRechitChamberPlus11 + {C}NRechitChamberMinus11 + {C}NRechitChamberPlus12 + {C}NRechitChamberMinus12")
    rdf = rdf.Define(f"{D}MB1", f"{D}MB1")
    # rdf = rdf.Define(f"{C}DNN", f"Take({C}DNN_bkgMC_plusBeamHalo, nCscRechitClusters)")

    # rdf = rdf.Define("pass_csc_gllp", csc_gllp_req)
    # rdf = rdf.Define("pass_dt_gllp", dt_gllp_req)

    rdf = rdf.Define("pass_hlt_csccsc", "HLTDecision[566]")
    rdf = rdf.Define("pass_hlt_cscdt", "HLTDecision[569]")

    rdf = rdf.Define("pass_csc_l1_pl", l1_plateau_req)
    rdf = rdf.Define("pass_csc_me1", f"{C}ME1 == 0")
    rdf = rdf.Define("pass_dt_mb1", f"{D}MB1 == 0")

    rdf = rdf.Define("pass_csc_time", f"(-5 < {C}TimeWeighted) && ({C}TimeWeighted < 12.5)")
    rdf = rdf.Define("pass_csc_tspread", f"{C}TimeSpreadWeightedAll < 20")

    rdf = rdf.Define("pass_csc_jet", f"{C}JetVetoPt < {MAX_CSC_JET_PT}")
    rdf = rdf.Define("pass_dt_jet", f"{D}JetVetoPt < {MAX_DT_JET_PT}")
    rdf = rdf.Define("pass_csc_muon", f"!( ({C}MuonVetoGlobal>0) && ({C}MuonVetoPt>{MAX_CSC_MUON_PT}) )")
    rdf = rdf.Define("pass_dt_muon", f"!( ({D}MuonVetoLooseId>0) && ({D}MuonVetoPt>{MAX_DT_MUON_PT}) )")

    # rdf = rdf.Define("pass_dnn", f"{C}DNN > {MIN_DNN}")


    # Now apply our trigger level selections
    rdf = rdf.Filter("(pass_hlt_csccsc || pass_hlt_cscdt) && (ROOT::VecOps::sum(pass_csc_me1 && pass_csc_l1_pl) > 0)")
    count, weight = rdf.Count(), rdf.Sum("weight")
    print(f"{key:>6}: N events = {count.GetValue():>10,} (weight = {weight.GetValue():>13,.2f})")

    rdfs[key] = rdf


In [2]:
# canvas = TCanvas("","",800,800)
# canvas.Divide(2, 1)
canvas = TCanvas("","",800,800)

form = (100, 0, 200)
for irdf, (key, rdf) in enumerate(rdfs.items()):
    canvas.cd(irdf + 1)
    h = rdf.Histo1D(("", ";Time Spread [ns];", *form), f"{C}TimeSpreadWeightedAll", "weight")
    # h.Scale(1/rdf.Sum("weight"))
    h.SetLineColor(std_color_list[irdf])
    h.Draw()
    gc.append(h)

canvas.Draw()

NameError: name 'rdfs' is not defined